# Import Modules

In [ ]:
import numpy as np
import pandas as pd

import utils

# Load Data

In [2]:
target_vegetable = "ミニトマト"

In [3]:
target_vegetables = [
    'だいこん', 'にんじん', 'キャベツ', 'レタス',
    'はくさい', 'こまつな', 'ほうれんそう', 'ねぎ',
    'きゅうり', 'トマト', 'ピーマン', 'じゃがいも',
    'なましいたけ', 'セルリー', 'そらまめ', 'ミニトマト'
]
target_vegetable = target_vegetables[3]

In [36]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
train_test = pd.concat([train, test])
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y%m%d")
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

train_test["weekday"] = train_test.date.dt.weekday
weekdays = pd.get_dummies(train_test["weekday"])
train_test = train_test.drop(columns="weekday")
train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

train = train_test[:-test.shape[0]]
test = train_test[-test.shape[0]:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

train_loader, test, ss = utils.preprocess_data(train, test, T=10)

# Training

In [ ]:
future = test.shape[0]
pred_y = utils.pipeline_rnn(train_loader, train, test, future=future, num_epochs=100)

pred_y = pred_y.detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :1] = ss.inverse_transform(pred_y[:, :1])

training loss = 1.019041895866394
training loss = 0.49156054854393005
training loss = 0.19074121117591858
training loss = 0.14833582937717438
training loss = 0.1398870199918747
training loss = 0.1253294199705124
training loss = 0.13166196644306183


In [ ]:
pred_y[:, 0].tolist()